# DLO-JZ Data Augmentation - Jour 4 

![car](./images/noun-car-repair-32305.png)



## Objet du notebook

Dans ce TP, on se focalise sur les **problématiques de performance liées à la *Data Augmentation***. 

Les opérations de transformation des données d'entrée se font en général sur le CPU. On verra dans ce TP qu'il est possible de les déléguer au GPU si les ressources de calcul offertes par le CPU ne sont pas suffisantes.

Ce TP est divisé en trois parties, correspondant à trois types d'augmentation de données à implémenter :

* **TP 1** : RandAugment sur CPU
* **TP 2** : mixup sur CPU et GPU
* **TP 3** : CutMix sur GPU


Les cellules dans ce *notebook* ne sont pas prévues pour être modifiées, sauf rares exceptions indiquées dans les commentaires. Les TP se feront en modifiant les codes `dlojz_da_X.py`.

Les directives de modification seront marquées par l'étiquette **TODO** dans le *notebook* suivant.
 
Les solutions sont présentes dans le répertoire `solutions/`.

*Notebook rédigé par l'équipe assistance IA de l'IDRIS, février 2024*

------------------------

### Environnement de calcul

Les fonctions *python* de gestion de queue SLURM dévelopées par l'IDRIS et les fonctions dédiées à la formation DLO-JZ sont à importer.

Le module d'environnement pour les *jobs* et la taille des images sont fixés pour ce *notebook*.

**TODO :** choisir un *pseudonyme* (maximum 5 caractères) pour vous différencier dans la queue SLURM et dans les outils collaboratifs pendant la formation et la compétition.

In [ ]:
from idr_pytools import display_slurm_queue, gpu_jobs_submitter, search_log
from dlojz_tools import controle_technique, compare, GPU_underthehood, plot_accuracy, lrfind_plot, imagenet_starter, turbo_profiler
MODULE = 'pytorch-gpu/py3/2.3.0'
account = 'for@a100'
name = 'pseudo'   ## Pseudonyme à choisir

------------------------------------

### Gestion de la queue SLURM

Pour afficher vos jobs dans la queue SLURM :

In [ ]:
display_slurm_queue(name)

**Remarque**: Cette fonction est utilisée plusieurs fois dans ce *notebook*. Elle permet d'afficher la queue de manière dynamique, rafraichie toutes les 5 secondes. Elle ne s'arrête que lorsque la queue est vide. Si vous désirez reprendre la main sur le *notebook*, il vous suffira d'arrêter manuellement la cellule avec le bouton *stop*. Cela a bien sûr aucun impact les jobs soumis.

Si vous voulez retirer TOUS vos jobs de la queue SLURM, décommenter et exécuter la cellule suivante :

In [ ]:
#!scancel -u $USER

Si vous voulez retirer UN de vos jobs de la queue SLURM, décommenter, compléter et exécuter la cellule suivante :

In [ ]:
#!scancel <jobid>

--------------

### Différence entre deux scripts

Pour comparer son code avec les solutions mises à disposition, la fonction suivante permet d'afficher une page html contenant un différentiel de fichiers texte.

In [ ]:
s1 = "dlojz_da_2.py"
s2 = "./solutions/dlojz_da_2.py"
#s1 = "mixup.py"
#s2 = "solutions/mixup-solution.py"
compare(s1, s2)

Voir le résultat du différentiel de fichiers sur la page suivante (attention au spoil !) :

[compare.html](compare.html)

----------------------
## Garage - Mise à niveau

On fixe le *batch size* et la taille d'image pour ce TP.

In [ ]:
bs_optim = 512
image_size = 224

----------------------------------

## TP_DA_1 : RandAugment

Le but de ce TP est d'ajouter la transformation `RandAugment` (disponible dans *torchvision*) dans la liste des transformations pour la *Data Augmentation* et de mesurer son impact sur la performance du code.

Voir la [documentation torchvision sur RandAugment](https://pytorch.org/vision/stable/generated/torchvision.transforms.RandAugment.html).

Vous pouvez exécuter les cellules suivantes pour observer l'effet de la transformation `RandAugment` :

In [ ]:
import os
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
import torch
import numpy as np
import matplotlib.pyplot as plt

transform = transforms.Compose([ 
        transforms.RandomResizedCrop(image_size),  # Random resize - Data Augmentation
        transforms.RandomHorizontalFlip(),  # Horizontal Flip - Data Augmentation
        transforms.RandAugment(5, 9),       # Random Augmentation 5: n operations, 9 : magnitude 
        transforms.ToTensor()               # convert the PIL Image to a tensor
        ])
    
    
train_dataset = torchvision.datasets.ImageNet(root=os.environ['ALL_CCFRSCRATCH']+'/imagenet',
                                                  transform=transform)
train_dataset

In [ ]:
%%time

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,    
                                           batch_size=4,
                                           shuffle=True)
batch = next(iter(train_loader))
print('X train batch, shape: {}, data type: {}, Memory usage: {} bytes'
      .format(batch[0].shape, batch[0].dtype, batch[0].element_size()*batch[0].nelement()))
print('Y train batch, shape: {}, data type: {}, Memory usage: {} bytes'
      .format(batch[1].shape, batch[1].dtype, batch[1].element_size()*batch[1].nelement()))

for i in range(4):
    img = batch[0][i].numpy().transpose((1,2,0))
    plt.imshow(img)
    plt.axis('off')
    plt.show()

### Transformation RandAugment sur CPU

**TODO :** dans le script [dlojz_da_1.py](dlojz_da_1.py) :
* Rajouter la transformation `RandAugment` dans la liste des transformations des images pour le *training* avec le paramétrage suivant : **Nombre d'opérations = 5, Magnitude = 9**.

Soumission du *job*. **Attention vous sollicitez les noeuds de calcul à ce moment-là**.

Pour soumettre le job, veuillez basculer la cellule suivante du mode `Raw NBConvert` au mode `Code`.

Copier-coller la sortie `jobid = ['xxxxx']` dans la cellule suivante.

Puis, rebasculer la cellule précédente en mode `Raw NBConvert`, afin d'éviter de relancer un job par erreur.

In [ ]:
#jobid = ['887754']

In [ ]:
display_slurm_queue(name)

In [ ]:
controle_technique(jobid)

In [ ]:
turbo_profiler(jobid)

![Commentaires](images/cedez.png "Assurez-vous que tout se passe bien avant de continuer!")

## TP_DA_2 : mixup

Le but de ce TP est de :
* appliquer la transformation `mixup` et mesurer son impact sur la performance du code ;
* porter la transformation sur GPU.

La transformation `mixup` n'est pas disponible dans *torchvision*, la fonction est disponible dans le script [mixup.py](mixup.py). On notera que cette transformation impacte à la fois l'image et le *label*.

On choisira, comme cela est fait habituellement, de *mixer* 2 images présentes dans le *batch* généré par le *DataLoader*. Donc cette transformation sera faite dans la boucle d'apprentissage après génération du *batch* et après toute autre transformation liée à la *Data Augmentation*.

Vous pouvez exécuter les cellules suivantes pour observer l'effet de la transformation `mixup` :

In [ ]:
import os
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
import torch
import numpy as np
import matplotlib.pyplot as plt

transform = transforms.Compose([ 
        transforms.RandomResizedCrop(image_size),  # Random resize - Data Augmentation
        transforms.RandomHorizontalFlip(),  # Horizontal Flip - Data Augmentation
        transforms.ToTensor()               # convert the PIL Image to a tensor
        ])
    
    
train_dataset = torchvision.datasets.ImageNet(root=os.environ['ALL_CCFRSCRATCH']+'/imagenet',
                                                  transform=transform)
train_dataset

In [ ]:
from mixup import mixup_data

In [ ]:
%%time

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,    
                                           batch_size=16,
                                           shuffle=True)
batch = next(iter(train_loader))
print('X train batch, shape: {}, data type: {}, Memory usage: {} bytes'
      .format(batch[0].shape, batch[0].dtype, batch[0].element_size()*batch[0].nelement()))
print('Y train batch, shape: {}, data type: {}, Memory usage: {} bytes'
      .format(batch[1].shape, batch[1].dtype, batch[1].element_size()*batch[1].nelement()))

imgs, targets = batch
imgs, targets = mixup_data(imgs, targets, num_classes=1000, alpha=2)        ## Transformation mixup

for i in range(4):
    img = imgs[i].numpy().transpose((1,2,0))
    plt.imshow(img)
    plt.axis('off')
    plt.show()
    print(f'target : {torch.max(targets, dim=1)[1][i]}, lambda : {torch.max(targets, dim=1)[0][i]}')

**Paramètre alpha pour la beta distribution**

Dans le script `mixup.py`, la variable `_lambda` correspond à la proportion conservée de la première image. Elle est choisie aléatoirement suivant une **distribution bêta** définie sur [0, 1].

Le paramètre `alpha` agit sur la forme de la distribution bêta. `alpha = 1` correspond à une distribution uniforme, `alpha < 1` favorise un tirage au sort de valeurs proches des bornes `0.` ou `1.`, et  `alpha > 1` favorise un tirage au sort de valeurs proches du centre `0.5`.

In [ ]:
for alpha in [0.5, 1., 2.]:
    plt.hist(np.random.beta(alpha, alpha, 1000000), bins=50, density=True, histtype='step')
    plt.title(f'alpha={alpha}')
    plt.show()

### Transformation mixup sur CPU

**TODO :** dans le script [dlojz_da_2.py](dlojz_da_2.py) :
* Importer la transformation `mixup`
```python
from mixup import mixup_data
```

* Rajouter la transformation `mixup` dans la boucle d'apprentissage **avant** d'envoyer le *batch* d'images et de *labels* au GPU, avec le paramétrage : **num_classes=1000, alpha=2**.


Soumission du *job*. **Attention vous sollicitez les noeuds de calcul à ce moment-là**.

Pour soumettre le job, veuillez basculer la cellule suivante du mode `Raw NBConvert` au mode `Code`.

Copier-coller la sortie `jobid = ['xxxxx']` dans la cellule suivante.

Puis, rebasculer la cellule précédente en mode `Raw NBConvert`, afin d'éviter de relancer un job par erreur.

In [ ]:
#jobid = ['887894']

In [ ]:
display_slurm_queue(name)

In [ ]:
controle_technique(jobid)

In [ ]:
turbo_profiler(jobid)

### Transformation mixup sur GPU

**TODO :** dans le script [dlojz_da_2.py](dlojz_da_2.py) :
* Appliquer la transformation `mixup` dans la boucle d'apprentissage **après** avoir envoyé le *batch* d'images et de *labels* au GPU, avec le paramétrage : **num_classes=1000, alpha=2, device=gpu**.

**TODO :** dans le script [mixup.py](mixup.py) :
* Ajouter le paramètre `device=device` à chaque fois que l'on crée un nouveau *Tensor* pour qu'il soit stocké en mémoire au bon emplacement (CPU ou GPU).

Soumission du *job*. **Attention vous sollicitez les noeuds de calcul à ce moment-là**.

Pour soumettre le job, veuillez basculer la cellule suivante du mode `Raw NBConvert` au mode `Code`.


Copier-coller la sortie `jobid = ['xxxxx']` dans la cellule suivante.

Puis, rebasculer la cellule précédente en mode `Raw NBConvert`, afin d'éviter de relancer un job par erreur.

In [ ]:
#jobid = ['887909']

In [ ]:
display_slurm_queue(name)

In [ ]:
controle_technique(jobid)

In [ ]:
turbo_profiler(jobid)

![Commentaires](images/cedez.png "Assurez-vous que tout se passe bien avant de continuer!")

## TP_DA_3 : CutMix

Le but de ce TP est de :
* appliquer la transformation `CutMix` et mesurer son impact sur la performance du code ;
* adapter l'implémentation de la tranformation `CutMix` au calcul GPU.

La transformation `CutMix` n'est pas disponible dans *torchvision*, la fonction est disponible dans le script [cutmix.py](cutmix.py). On notera que cette transformation impacte à la fois l'image et le *label*.

On choisira, comme cela est fait habituellement, de *mixer* 2 images présentes dans le *batch* généré par le dataloader. Donc cette transformation sera faite dans la boucle d'apprentissage après génération du *batch* et donc après toute autre transformation liée à la *Data Augmentation*.

Dans le script `cutmix.py`, la variable `_lambda` correspond à la proportion conservée de la première image. Elle est choisie aléatoirement suivant une **distribution uniforme** définie sur [0, 1].

In [ ]:
import os
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
import torch
import numpy as np
import matplotlib.pyplot as plt

transform = transforms.Compose([ 
        transforms.RandomResizedCrop(image_size),  # Random resize - Data Augmentation
        transforms.RandomHorizontalFlip(),  # Horizontal Flip - Data Augmentation
        transforms.ToTensor()               # convert the PIL Image to a tensor
        ])
    
    
train_dataset = torchvision.datasets.ImageNet(root=os.environ['ALL_CCFRSCRATCH']+'/imagenet',
                                                  transform=transform)
train_dataset

In [ ]:
from cutmix import cutmix_data

In [ ]:
%%time

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,    
                                           batch_size=16,
                                           shuffle=True)
batch = next(iter(train_loader))
print('X train batch, shape: {}, data type: {}, Memory usage: {} bytes'
      .format(batch[0].shape, batch[0].dtype, batch[0].element_size()*batch[0].nelement()))
print('Y train batch, shape: {}, data type: {}, Memory usage: {} bytes'
      .format(batch[1].shape, batch[1].dtype, batch[1].element_size()*batch[1].nelement()))

imgs, targets = batch
imgs, targets = cutmix_data(imgs, targets, num_classes=1000)

for i in range(4):
    img = imgs[i].numpy().transpose((1,2,0))
    plt.imshow(img)
    plt.axis('off')
    plt.show()
    print(f'target : {torch.max(targets, dim=1)[1][i]}, lambda : {torch.max(targets, dim=1)[0][i]}')

### Transformation CutMix sur GPU

**TODO :** dans le script [dlojz_da_3.py](dlojz_da_3.py) :
* Importer la transformation `CutMix`
```python
from cutmix import cutmix_data
```

* Rajouter la transformation `CutMix` dans la boucle d'apprentissage **après** avoir envoyé le *batch* d'images et de *labels* au GPU, avec le paramétrage : **num_classes=1000, device=gpu**.

Soumission du *job*. **Attention vous sollicitez les noeuds de calcul à ce moment-là**.

Pour soumettre le job, veuillez basculer la cellule suivante du mode `Raw NBConvert` au mode `Code`.


Copier-coller la sortie `jobid = ['xxxxx']` dans la cellule suivante.

Puis, rebasculer la cellule précédente en mode `Raw NBConvert`, afin d'éviter de relancer un job par erreur.

In [ ]:
#jobid = ['887951']

In [ ]:
display_slurm_queue(name)

In [ ]:
controle_technique(jobid)

In [ ]:
turbo_profiler(jobid)

### Optimisation de la transformation CutMix
Le code précédent traite les images du *batch* une par une, de manière séquentielle (boucle `for`) :

```python
mixed_x = x
for i in range(len(mixed_x)): # loop over images
            mixed_x[i,:,x1[i]:x2[i],y1[i]:y2[i]] = x[s_index[i],:,x1[i]:x2[i],y1[i]:y2[i]]
```

Le but de cette partie est d'optimiser le code de *CutMix* en générant davantage de parallélisme pour profiter du GPU. Il s'agit de supprimer la boucle `for` et de manipuler directement des batches de tenseurs.

Le travail va porter sur la définition de deux **batches de masques** `mask_int` et `mask_ext` de taille `[batch_size,n_channels,weight,height]` que l'on appliquera de la manière suivante : 

```python
mixed_x = mask_ext * x + mask_int * x[s_index, :]
```

La fonction à implémenter est la suivante :
```python
def cut_mask(x1, x2, y1, y2, batch_size, W, H, device=None):
    
    ### TODO
    mask_ext, mask_int = None, None
    
    return mask_ext, mask_int
```

Arguments :
* `x1` : vecteur de longueur `batch_size` avec la coordonnée **min** dans la **largeur** pour chaque image du batch
* `x2` : vecteur de longueur `batch_size` avec la coordonnée **max** dans la **largeur** pour chaque image du batch
* `y1` : vecteur de longueur `batch_size` avec la coordonnée **min** dans la **hauteur** pour chaque image du batch
* `y2` : vecteur de longueur `batch_size` avec la coordonnée **max** dans la **hauteur** pour chaque image du batch
* `batch_size` : nombre d'images dans le batch
* `W` : largeur des images
* `H` : hauteur des images
* `device` : unité de calcul ('cpu' ou 'gpu')

Retours:
* `mask_ext` : tenseur de taille `[batch_size,n_channels,weight,height]` contenant la valeur `False` ou `0` à l'intérieur de la fenêtre et `True` ou `1` à l'extérieur
* `mask_int` : tenseur de taille `[batch_size,n_channels,weight,height]` contenant la valeur `True` ou `1` à l'intérieur de la fenêtre et `False` ou `0` à l'extérieur


**Création d'un batch de masques**

Dans un premier temps, pour comprendre la procédure, nous travaillerons avec un *batch* de 3 images de taille `32x32`.

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
batch_size = 3
W = 32
H = 32

En entrée, on connait les coordonnées des coins de la fenêtre pour chaque image du *batch* (voir illustration ci-dessous). 

In [ ]:
# coordonnee min dans la largeur pour chaque image du batch
x1 = torch.Tensor([10, 5, 23]).int()
# coordonne max dans la largeur pour chaque image du batch
x2 =  torch.Tensor([20, 25, 31]).int()
# coordonnee min dans la hauteur pour chaque image du batch
y1 =  torch.Tensor([5, 10, 0]).int()
# coordonne max dans la hauteur pour chaque image du batch
y2 =  torch.Tensor([10, 22, 20]).int()

<div><img src="./images/cutmix_opt.png" width="500"/></div>

**1. Création de w_int et h_int**

Pour construire `mask_int`, on va d'abord créer un batch de vecteurs ligne "largeur" `w_int` et un batch de vecteurs colonne "hauteur" de masques `h_int` (voir illustration ci-dessus).

Variables utiles : `batch_size`, `W`, `H`, `x1`, `x2`, `y1`, `y2`.

Voir la documentation PyTorch pour la manipulation de tenseurs : [documentation torch](https://pytorch.org/docs/stable/index.html).

**Résultats attendus :** (voir illustration ci-dessous)
* un batch de vecteurs ligne "largeur" `w_int`, tenseur de taille `[3, 1, 32]` contenant des `True` ou `1` si **`x1 ⩽ x ⩽ x2`**, `False` ou `0` sinon 
* un batch de vecteurs colonne "hauteur" `h_int`, tenseur de taille `[3, 32, 1]` contenant des `True` ou `1` si **`y1 ⩽ y ⩽ y2`**, `False` ou `0` sinon 
![résultat](images/resultat_cutmix.png "w_int torch.Size([3, 1, 32]) / h_int torch.Size([3, 32, 1])")

<details>

<summary>Pistes de solutions</summary>
    
Nous avons trouvé 2 solutions pour résoudre ce problème.

* En utilisant la fonction [torch.logical_and](https://pytorch.org/docs/stable/generated/torch.logical_and.html) : il s'agit d'initialiser des tenseurs à `[x, x=0,...,31]` (respectivement `[y, y=0,...,31]`) et d'utiliser `torch.logical_and()` pour appliquer les conditions `x1 ⩽ x and x ⩽ x2` (respectivement `y1 ⩽ y and y ⩽ y2`). Le résultat est un tenseur contenant des opérateurs logiques `True` et `False`.

* En utilisant la fonction [torch.cumsum](https://pytorch.org/docs/stable/generated/torch.cumsum.html) : les tenseurs sont initialisés à `0`, on donne la valeur `1` au `x1`ème élément (respectivement `y1`ème), la valeur `-1` au `x2`ème élément (respectivement `y2`ème), puis on utilise la fonction `torch.cumsum` pour faire la somme cumulée des éléments. On obtient un tenseur contenant des `0` et des `1`.

Dans tous les cas, il faudra jouer avec les dimensions des tenseurs. Les fonctions utiles sont : [torch.arange()](https://pytorch.org/docs/stable/generated/torch.arange.html), [.size()](https://pytorch.org/docs/stable/generated/torch.Tensor.size.html), [.repeat()](https://pytorch.org/docs/stable/generated/torch.Tensor.repeat.html#torch.Tensor.repeat), [.view()](https://pytorch.org/docs/stable/generated/torch.Tensor.view.html#torch.Tensor.view), [.unsqueeze()](https://pytorch.org/docs/stable/generated/torch.unsqueeze.html#torch-unsqueeze), [.zeros()](https://pytorch.org/docs/stable/generated/torch.zeros.html), ...
    
Il existe certainement d'autres solutions.
    
**Rappel** : ne jamais utiliser de `for` nulle part !
    
</details>

In [ ]:
w_int = #TODO
# assert w_int has the correct size
assert w_int.size() == torch.Size([3,1,32])

In [ ]:
for wx in w_int:
    plt.imshow(wx)
    plt.colorbar()
    plt.show()

In [ ]:
h_int = #TODO
# assert h_int has the correct size
assert h_int.size() == torch.Size([3,32,1])                              

In [ ]:
for hx in h_int:
    plt.imshow(hx)
    plt.colorbar()
    plt.show()

<details>

<summary>Solution 1 avec <code>torch.logical_and</code></summary>    

<br>

```python
# initialisation du tenseur w_int avec les valeurs [0,...,31]
w_int = torch.arange(W).repeat(batch_size,1,1) # batch de vecteurs ligne 
# Returns the mask applying ((x1 ⩽ x) and (x ⩽ x2))
w_int = torch.logical_and(w_int >= x1.view(-1,1,1), w_int <= x2.view(-1,1,1)) # vecteurs ligne

# initialisation du tenseur h_int avec les valeurs [0,...,31]
h_int = torch.arange(H).repeat(batch_size,1).unsqueeze(2) # batch de vecteurs colonne
# Returns the mask applying ((y1 ⩽ y) and (y ⩽ y2))
h_int = torch.logical_and(h_int >= y1.view(-1,1,1), h_int <= y2.view(-1,1,1)) # vecteurs colonne
```
</details>

<details>

<summary>Solution 2 avec <code>torch.cumsum</code></summary>
    
<br>

On initialise les éléments correspondant aux coordonnées `x1` et `y1` à `1`. <br>
On initialise les éléments correspondant aux coordonnées `x2` et `y2` à `-1`. <br>
Puis on utilise la fonction [torch.cumsum](https://pytorch.org/docs/stable/generated/torch.cumsum.html) pour remplir chaque intervalle `[x1,x2]` et `[y1,y2]` de `1`, le reste de `0`.
    
**Remarque :** il y a une petite erreur dans cette solution qui n'a pas d'impact majeur. +1 sur votre appréciation finale si vous la trouvez !!    
    
```python
w_int = torch.zeros(batch_size,1,W) # vecteurs ligne 
w_int[range(batch_size),0,x1] = 1.
w_int[range(batch_size),0,x2] = -1.
w_int = torch.cumsum(w_int, dim=2).bool() # vecteurs ligne

h_int = torch.zeros(batch_size,H,1) # vecteurs colonne
h_int[range(batch_size),y1,0] = 1.
h_int[range(batch_size),y2,0] = -1.
h_int = torch.cumsum(h_int, dim=1).bool() # vecteurs colonne
```
<br> 
<details>
    
<summary>Solution +++</summary>

<br>
    
Avec la solution précédente, les bornes `x2` et `y2` sont exclues de la fenêtre. Pour les inclure, il faudrait définir la valeur `-1` sur les `x2+1`ème et `y2+1`ème éléments :

```python
w_int[range(batch_size),0,x2+1] = -1.
h_int[range(batch_size),y2+1,0] = -1.   
```

Cela entraîne des cas particuliers si `x2=31` ou `y2=31`. Pour gérer ces exceptions sans introduire de `if` :
    
```python
# if x2==31, set w_int(.,0,31)=0, otherwize set w_int(.,0,x2+1)=-1
w_int[.,0,torch.minimum(torch.tensor([31]).repeat(batch_size),x2+1)]=torch.maximum(torch.tensor([-1.]).repeat(batch_size),x2-31)
# if y2==31, set h_int(.,31,0)=0, otherwize set h_int(.,y2+1,0)=-1
h_int[.,torch.minimum(torch.tensor([31]).repeat(batch_size),y2+1,0)]=torch.maximum(torch.tensor([-1.]).repeat(batch_size),y2-31)
```
    
</details>
    

</details>

**2. Création des batches de masques intérieurs et extérieurs**

* Multiplication des vecteurs h_int et w_int pour obtenir les masques intérieurs pour chaque image du batch.

In [ ]:
# multiplication des vecteurs colonne "hauteur" h_int par les vecteurs ligne "largeur" w_int
mask_int = #TODO

In [ ]:
# visualisation des masques intérieurs pour chaque image du batch
for m in mask_int:
    plt.imshow(m)
    plt.colorbar()
    plt.show()

<details>

<summary>Solution</summary>

```python
# multiplication des vecteurs colonne "hauteur" h_int par les vecteurs ligne "largeur" w_int
mask_int = h_int*w_int
```
</details>

* Puis création des masques extérieurs à partir des masques intérieurs.


<details>

<summary>Aide</summary>
    
Par exemple en utilisant la fonction [torch.logical_not](https://pytorch.org/docs/stable/generated/torch.logical_not.html).

</details>

In [ ]:
# les masques extérieurs sont les complémentaires des masques intérieurs
mask_ext = #TODO

In [ ]:
# visualisation des masques extérieurs
for m in mask_ext:
    plt.imshow(m)
    plt.colorbar()
    plt.show()

<details>

<summary>Solution</summary>

```python
# les masques extérieurs sont les complémentaires des masques intérieurs
mask_ext = torch.logical_not(mask_int)
```
</details>

**Implémentation de la fonction de création d'un batch de masques**

Maintenant, l'idée est d'implémenter ce qui a été fait dans les cellules précédentes dans une fontion générique, en ajoutant un choix sur le *device* d'exécution.

**TODO** : implémenter la fonction de création des masques dans la cellule suivante. Les entrées de la fonction sont : 
* les coordonnées `x1`, `x2`, `y1`, `y2`, 
* le `batch_size`, 
* la largeur`W` de l'image, 
* la hauteur `H` de l'image, 
* le `device` de calcul.

**Important** : Pour les images RGB (*channel* de 3), il faut rajouter une dimension en deuxième position dans les masques finaux ([doc .unsqueeze()](https://pytorch.org/docs/stable/generated/torch.unsqueeze.html#torch-unsqueeze)) :
```python
    # rajouter une dimension en 2e position pour pouvoir traiter des images RGB
    mask_int = mask_int.unsqueeze(1) 
    mask_ext = mask_ext.unsqueeze(1) 
```

**Attention** : Ne pas oublier le paramètre `device=device` à chaque création d'un nouveau *Tensor*. Par exemple pour :
```python
    w_int = torch.zeros(batch_size,1,W,device=device)
```


In [ ]:
def cut_mask(x1, x2, y1, y2, batch_size, W, H, device=None):
    
    #TODO
    
    return mask_ext, mask_int

### Test de la fonction implémentée

In [ ]:
%%time

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,    
                                           batch_size=16,
                                           shuffle=True)
batch = next(iter(train_loader))
print('X train batch, shape: {}, data type: {}, Memory usage: {} bytes'
      .format(batch[0].shape, batch[0].dtype, batch[0].element_size()*batch[0].nelement()))
print('Y train batch, shape: {}, data type: {}, Memory usage: {} bytes'
      .format(batch[1].shape, batch[1].dtype, batch[1].element_size()*batch[1].nelement()))

imgs, targets = batch

In [ ]:
batch_size = 16
W = image_size
H = image_size

In [ ]:
lam = torch.rand(batch_size)
s_index = torch.randperm(batch_size)      # Shuffle index
rand_x = torch.randint(W, (batch_size,))
rand_y = torch.randint(H, (batch_size,))
cut_rat = torch.sqrt(1. - lam) ## cut ratio according to the random lambda

x1 = torch.clip(rand_x - rand_x / 2, min=0).long()
x2 = torch.clip(rand_x + rand_x / 2, max=W-1).long()
y1 = torch.clip(rand_y - rand_y / 2, min=0).long()
y2 = torch.clip(rand_y + rand_y / 2, max=H-1).long()

mask_ext, mask_int = cut_mask(x1, x2, y1, y2, batch_size, W, H)

In [ ]:
# vérifier si le masque intérieur et l'image ont le même nombre de dimensions
try:
    assert imgs.dim() == mask_int.dim()
    print('OK!')
except:
    print(f'Mismatch: \n dim imgs = {imgs.dim()} \n dim mask int = {mask_int.dim()} ')

In [ ]:
# vérifier si le masque extérieur et l'image ont le même nombre de dimensions
try:
    assert imgs.dim() == mask_ext.dim()
    print('OK!')
except:
    print(f'Mismatch: \n dim imgs = {imgs.dim()} \n dim mask ext = {mask_ext.dim()} ')

In [ ]:
imgs = mask_ext * imgs + mask_int * imgs[s_index, :]

In [ ]:
for i in range(4):
    img = imgs[i].numpy().transpose((1,2,0))
    plt.imshow(img)
    plt.axis('off')
    plt.show()

Puis si le résultat est satisfaisant, intégrer la fonction dans le code [cutmix.py](cutmix.py).

### Intégration de la nouvelle version dans cutmix.py

**TODO :** dans le script [cutmix.py](cutmix.py), ajouter la fonction `cut_mask` définie dans la cellule plus haut.

Soumission du *job*. **Attention vous sollicitez les noeuds de calcul à ce moment-là**.

Pour soumettre le job, veuillez basculer la cellule suivante du mode `Raw NBConvert` au mode `Code`.

Copier-coller la sortie `jobid = ['xxxxx']` dans la cellule suivante.

Puis, rebasculer la cellule précédente en mode `Raw NBConvert`, afin d'éviter de relancer un job par erreur.

In [ ]:
#jobid = ['888412']

In [ ]:
display_slurm_queue(name)

In [ ]:
controle_technique(jobid)

In [ ]:
turbo_profiler(jobid)

![Garage](images/stop.png "Arrêtez-vous ici! Une présentation vous attend avant le prochain TP.")